In [10]:
import numpy as np
import pandas as pd
import matplotlib as plt
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression


from main import Preprocessing, Pipe

In [11]:
! ls

README.md    __pycache__  main.py      pipe.ipynb   turnover.csv


In [12]:
data = pd.read_csv('turnover.csv', encoding = 'ISO-8859-1')

In [13]:
data

,stag,event,gender,age,industry,profession,traffic,coach,head_gender,greywage,way,extraversion,independ,selfcontrol,anxiety,novator
0,7.030801,1,m,35.0,Banks,HR,rabrecNErab,no,f,white,bus,6.2,4.1,5.7,7.1,8.3
1,22.965092,1,m,33.0,Banks,HR,empjs,no,m,white,bus,6.2,4.1,5.7,7.1,8.3
2,15.934292,1,f,35.0,PowerGeneration,HR,rabrecNErab,no,m,white,bus,6.2,6.2,2.6,4.8,8.3
3,15.934292,1,f,35.0,PowerGeneration,HR,rabrecNErab,no,m,white,bus,5.4,7.6,4.9,2.5,6.7
4,8.410678,1,m,32.0,Retail,Commercial,youjs,yes,f,white,bus,3.0,4.1,8.0,7.1,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,10.611910,0,f,41.0,Banks,HR,rabrecNErab,my head,m,white,bus,8.6,3.4,2.6,4.8,8.3
1125,10.611910,0,f,41.0,Banks,HR,rabrecNErab,my head,m,white,bus,8.6,3.4,2.6,4.8,8.3
1126,118.800821,0,f,34.0,Telecom,Accounting,KA,no,f,white,bus,4.6,5.5,7.2,6.3,3.7
1127,49.412731,0,f,51.0,Consult,HR,empjs,no,m,grey,bus,3.8,7.6,5.7,6.3,5.2


In [14]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('event', axis=1),data['event'], test_size=0.3)

In [6]:
categories = ['gender', 'industry', 'profession',
              'traffic', 'coach', 'head_gender', 'greywage', 'way']
numeric = list(set(X_train.columns) - set(categories))

In [7]:
lr = Pipe(LogisticRegression(),
          is_linear=True)
params = {'C' : [1.0, 0.5, 0.2], 'max_iter' : [100,200,500]}
lr.fit(X_train, y_train, params, categories, numeric)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
-----------------------------------------------
best_params is {'C': 0.5, 'max_iter': 100}
best_score is 0.6601193308107265



LogisticRegression(C=0.5)

In [9]:
lgbm = Pipe(LGBMClassifier(),
            is_linear=False)
params = {'learning_rate' : [0.1, 0.5, 1], 'n_estimators' : [100, 500, 1000], 'max_depth' : [5,10,15,30]}
lgbm.fit(X_train, y_train, params, categories, numeric)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
-----------------------------------------------
best_params is {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100}
best_score is 0.7122942832550668



LGBMClassifier(max_depth=10)